Load libraries

In [1]:
from fastai.vision import *

Look at dataset

In [4]:
! wget https://www.kaggle.com/alxmamaev/flowers-recognition/downloads/flowers.zip

--2019-02-13 20:12:14--  https://www.kaggle.com/alxmamaev/flowers-recognition/downloads/flowers.zip
Resolving www.kaggle.com (www.kaggle.com)... 23.96.207.25
Connecting to www.kaggle.com (www.kaggle.com)|23.96.207.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?returnUrl=%2Falxmamaev%2Fflowers-recognition%2Fdata [following]
--2019-02-13 20:12:14--  https://www.kaggle.com/account/login?returnUrl=%2Falxmamaev%2Fflowers-recognition%2Fdata
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘flowers.zip’

flowers.zip             [ <=>                ]   7.81K  --.-KB/s    in 0.001s  

2019-02-13 20:12:14 (8.30 MB/s) - ‘flowers.zip’ saved [7997]



In [ ]:
path = Path('../input/flowers/flowers')

In [3]:
path.ls()

NameError: name 'path' is not defined

In [ ]:
get_image_files(path/'daisy')[:5]

Create datablock

In [ ]:
bs = 64
size = 224
num_workers = 0  # set this to 0 to prevent kernel from crashing

In [ ]:
tfms = get_transforms()                               #Do standard data augmentation
data = (ImageItemList.from_folder(path)               #Get data from path
        .random_split_by_pct()                        #Randomly separate 20% of data for validation set
        .label_from_folder()                          #Label based on dir names
        .transform(tfms, size=size)                   #Pass in data augmentation
        .databunch(bs=bs, num_workers=num_workers)    #Create ImageDataBunch
        .normalize(imagenet_stats))                   #Normalize using imagenet stats

Verify datasets loaded properly.  We should have the following:
* train: 3,459
* valid: 864

In [ ]:
print(len(data.train_ds))
print(len(data.valid_ds))

In [ ]:
data.classes

Visualize data

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

Create learner using pre-trained model

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, model_dir='/tmp/models')

Fit the model with most layers frozen

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

Unfreeze all layers, use learning rate finder, fit some more

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('stage-2')

Examine prediction errors in validation set

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(9,9), dpi=60)

In [ ]:
interp.most_confused(min_val=2)